In [2]:
from cpg_fsm import *
from scipy.optimize import differential_evolution
from scipy.optimize import LinearConstraint, Bounds
from scipy.optimize import minimize
import pickle


In [3]:
loop_freq = 100  # Hz

L = 0.2 # step_length
T_sw = 0.3
H_upper = 0.055
C_s = 3
C_u = 0.5
px_init = 0.2/8
mx_init = -0.2/8
cycle = 4*6
total_t = 30

dH_upper = 0.02
L1_lower = 0.01
L3_lower = 0.01

tb_1 = lk.InverseKinematicsPoly(np.array([[px_init], [-0.2]]))
tb_2 = lk.InverseKinematicsPoly(np.array([[mx_init], [-0.2]]))
init_A_tb = tb_1
init_B_tb = tb_1
init_C_tb = tb_2
init_D_tb = tb_2

FSM = FiniteStateMachine(loop_freq)
CORGI = Corgi(loop_freq, FSM)
CORGI.step_length = L
CORGI.swing_time = T_sw
CORGI.weight_s = C_s
CORGI.weight_u = C_u
CORGI.total_cycle = cycle
CORGI.total_time = total_t
CORGI.setInitPhase(init_A_tb, init_B_tb, init_C_tb, init_D_tb)
CORGI.move()
CORGI.exportCSV("/home/guanlunlu/corgi_webots/controllers/supervisor/output.csv")


t = time.time()
iter = 0

cost -9.59643447771501


In [4]:
cons_params = [] #[B14_x, B910_x] for each leg
c_mats = []
up_vecs = []
lo_vecs = []

r = 0.5 # wide --> 1
for i in range(4):
    v_OG = CORGI.lift_state[i][2]
    if i == 0 or i == 3:
        v_OG[0,0] *= -1
    B_14 = -v_OG[0,0] - 0.222 - r * L/4
    B_910 = -v_OG[0,0] + 0.222 + r * L/4
    cons_params.append([B_14, B_910])

    c_mat = np.array([[1, 0, 0, 0, 0, 0,
                       0, 1, 0, 0, 0, 0,
                       0, 0, -1, -0.5, 0, 0,
                       0, 0, -1, -1, 0, 0,
                       0, 0, 0, 0, 1, 0.5,
                       0, 0, 0, 0, 1, 1,
                       0, 0, 1, 0, 0, 0,
                       0, 0, 0, 0, 1, 0]])
    c_mat.resize(8, 6)
    c_mats.append(c_mat)
    v_up = np.array([[H_upper], [dH_upper], [np.inf], [0.5*L], [B_910], [np.inf], [np.inf], [np.inf]])
    v_lo = np.array([[0], [0], [B_14], [-np.inf], [-np.inf], [-0.5*L], [L1_lower], [L3_lower]])
    up_vecs.append(v_up)
    lo_vecs.append(v_lo)

# Define Constraint Function
z_ = np.zeros([8, 6])    
r1 = np.hstack((c_mats[1], z_, z_, z_))
r2 = np.hstack((z_, c_mats[1], z_, z_))
r3 = np.hstack((z_, z_, c_mats[1], z_))
r4 = np.hstack((z_, z_, z_, c_mats[1]))
cons_mat = np.vstack((r1, r2, r3, r4))
upper_bound = np.vstack((up_vecs[0], up_vecs[1], up_vecs[2], up_vecs[3]))
lower_bound = np.vstack((lo_vecs[0], lo_vecs[1], lo_vecs[2], lo_vecs[3]))
upper_bound = upper_bound.T[0]
lower_bound = lower_bound.T[0]
constraint = LinearConstraint(cons_mat, lower_bound, upper_bound)

In [5]:
def getTrajectoryCost(bezier_profiles):
    global iter
    global t
    print("t", time.time()-t)
    
    t = time.time()
    bp = bezier_profiles.reshape(4, -1)
    tb_1 = lk.InverseKinematicsPoly(np.array([[px_init], [-0.2]]))
    tb_2 = lk.InverseKinematicsPoly(np.array([[mx_init], [-0.2]]))
    init_A_tb = tb_1
    init_B_tb = tb_1
    init_C_tb = tb_2
    init_D_tb = tb_2

    FSM = FiniteStateMachine(loop_freq)
    CORGI = Corgi(loop_freq, FSM)
    CORGI.total_cycle = 4
    CORGI.step_length = L
    CORGI.swing_time = T_sw
    CORGI.weight_s = C_s
    CORGI.weight_u = C_u
    CORGI.setInitPhase(init_A_tb, init_B_tb, init_C_tb, init_D_tb)
    CORGI.move(swing_profile=bp)
    
    print("iter", iter,"cost",CORGI.cost)
    iter += 1
    return CORGI.cost

In [6]:
bnd_H = (0.05, H_upper)
bnd_dH = (0, dH_upper)
f_bnd_L1 = (L1_lower, 0.5)
f_bnd_L2 = (-0.5, 0.5)
f_bnd_L3 = (L3_lower, 0.5)
f_bnd_L4 = (-0.5, 0.5)

h_bnd_L1 = (L1_lower, 0.5)
h_bnd_L2 = (-0.5, 0.5)
h_bnd_L3 = (L3_lower, 0.5)
h_bnd_L4 = (-0.5, 0.5)

bez_prof_init = np.array([[0.06, 0.01, 0.15, 0.05, 0.08, 0.06],
                          [0.06, 0.01, 0.15, 0.05, 0.08, 0.06],
                          [0.06, 0.01, 0.08, 0.05, 0.02, 0.1],
                          [0.06, 0.01, 0.08, 0.05, 0.02, 0.1]])

bez_prof_init = bez_prof_init.reshape(1, -1)[0]


In [7]:
# h, dh, dL1, dL2, dL3, dL4 = bezier_profile
bnds = [bnd_H, bnd_dH, f_bnd_L1, f_bnd_L2, f_bnd_L3, f_bnd_L4,
        bnd_H, bnd_dH, f_bnd_L1, f_bnd_L2, f_bnd_L3, f_bnd_L4,
        bnd_H, bnd_dH, h_bnd_L1, h_bnd_L2, h_bnd_L3, h_bnd_L4,
        bnd_H, bnd_dH, h_bnd_L1, h_bnd_L2, h_bnd_L3, h_bnd_L4]

bnd = [bnd_H, bnd_dH, f_bnd_L1, f_bnd_L2, f_bnd_L3, f_bnd_L4,
        bnd_H, bnd_dH, f_bnd_L1, f_bnd_L2, f_bnd_L3, f_bnd_L4,
        bnd_H, bnd_dH, h_bnd_L1, h_bnd_L2, h_bnd_L3, h_bnd_L4,
        bnd_H, bnd_dH, h_bnd_L1, h_bnd_L2, h_bnd_L3, h_bnd_L4]

# result = differential_evolution(getTrajectoryCost, bounds=bnds, recombination=0.5, constraints=constraint, seed=1, workers=-1)
# min_ = minimize(getTrajectoryCost, bez_prof_init, method='Nelder-Mead',bounds=bnd, constraints=[constraint],\
                #  options={'disp': True, 'maxiter': 800, "adaptive": True})
# min_ = minimize(getTrajectoryCost, bez_prof_init, bounds=bnd, constraints=[constraint], \
#         options={"maxiter": 200})


min_ = minimize(getTrajectoryCost, bez_prof_init, method="SLSQP", bounds=bnd,constraints=[constraint], options={'disp': True})

t 0.05647993087768555
cost -1.1766878453827558
iter 0 cost -1.1766878453827558
t 0.12270784378051758
cost -1.1766878489436363
iter 1 cost -1.1766878489436363
t 0.1236562728881836
cost -1.1766878464932342
iter 2 cost -1.1766878464932342
t 0.12295007705688477
cost -1.1766878149197704
iter 3 cost -1.1766878149197704
t 0.1269974708557129
cost -1.1766878257950404
iter 4 cost -1.1766878257950404
t 0.11954951286315918
cost -1.1766877984765853
iter 5 cost -1.1766877984765853
t 0.12298059463500977
cost -1.1766878248287702
iter 6 cost -1.1766878248287702
t 0.11523556709289551
cost -1.1766878413443027
iter 7 cost -1.1766878413443027
t 0.15186691284179688
cost -1.1766878436675523
iter 8 cost -1.1766878436675523
t 0.1329953670501709
cost -1.176687845915611
iter 9 cost -1.176687845915611
t 0.1225590705871582
cost -1.1766878500876545
iter 10 cost -1.1766878500876545
t 0.12138152122497559
cost -1.1766878354320316
iter 11 cost -1.1766878354320316
t 0.1180117130279541
cost -1.1766878460356656
iter 12 co

/home/guanlunlu/.local/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


cost -1.9099001374986035
iter 25 cost -1.9099001374986035
t 0.11929440498352051
cost -1.909900122509911
iter 26 cost -1.909900122509911
t 0.12430858612060547
cost -1.9099001544459693
iter 27 cost -1.9099001544459693
t 0.12364840507507324
cost -1.9099001376272118
iter 28 cost -1.9099001376272118
t 0.11708593368530273
cost -1.9099001377301992
iter 29 cost -1.9099001377301992
t 0.11547732353210449
cost -1.9099001248268515
iter 30 cost -1.9099001248268515
t 0.11453080177307129
cost -1.9099001416189516
iter 31 cost -1.9099001416189516
t 0.12138867378234863
cost -1.9099001295587028
iter 32 cost -1.9099001295587028
t 0.12067508697509766
cost -1.909900136631631
iter 33 cost -1.909900136631631
t 0.12104105949401855
cost -1.9099001332271375
iter 34 cost -1.9099001332271375
t 0.1193075180053711
cost -1.9099001372946494
iter 35 cost -1.9099001372946494
t 0.11428332328796387
cost -1.9099001352549871
iter 36 cost -1.9099001352549871
t 0.1162106990814209
cost -1.909900137824035
iter 37 cost -1.909900

Traceback (most recent call last):
  File "/home/guanlunlu/MBD_Corgi_python/script/cpg_fsm.py", line 351, in move
    self.legs[i].moveSwing(lf_sp)
  File "/home/guanlunlu/MBD_Corgi_python/script/cpg_fsm.py", line 136, in moveSwing
    tb_ = lk.InverseKinematicsPoly(sp)
  File "/home/guanlunlu/MBD_Corgi_python/script/LegKinematics.py", line 149, in InverseKinematicsPoly
    raise ValueError("Desired length exceed ", str(desired_length), "Point desired", str(point_G.T))
ValueError: ('Desired length exceed ', '0.34312323902125963', 'Point desired', '[[-0.30464381 -0.15787877]]')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_21027/3020735164.py", line 19, in <module>
    min_ = minimize(getTrajectoryCost, bez_prof_init, method="SLSQP", boun

In [8]:
# with open('min.pkl', 'rb') as f:
#     min_ = pickle.load(f)

opt_bez = min_.x
opt_bez = opt_bez.reshape(4,-1)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_21027/2353228091.py", line 4, in <module>
    opt_bez = min_.x
NameError: name 'min_' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPyth

In [9]:
# Save Parameters and export Trajectory
import datetime

sbrio_filename = str(datetime.date.today()) + "_traj_400_14.csv"
param_filename = str(datetime.date.today()) + "_param_400_14.csv"
csv_filepath = "./csv_trajectory/" + sbrio_filename
webot_filepath = "/home/guanlunlu/corgi_webots/controllers/supervisor/" + sbrio_filename
param_filepath = "./csv_trajectory/" + param_filename

with open(str(datetime.date.today()) + "_param_400_14.pkl", 'wb') as f:
    pickle.dump(min_, f)

with open(param_filepath, 'w') as f:
    f.write("loop_freq,"+str(loop_freq)+"\n")
    f.write("Step_length,"+str(L)+"\n")
    f.write("swing_time,"+str(T_sw)+"\n")
    f.write("C_s,"+str(C_s)+"\n")
    f.write("C_u,"+str(C_u)+"\n")
    f.write("px_init,"+str(px_init)+"\n")
    f.write("mx_init,"+str(mx_init)+"\n")
    f.write("cycle,"+str(cycle)+"\n")
    f.write("total_t,"+str(total_t)+"\n")
    f.write("init_bezier_profile"+repr(bez_prof_init)+"\n")
    f.write("optimize_bezier_profile"+repr(min_.x)+"\n")
    f.write("\n")
    f.write(repr(min_))
    f.write('\n')

cycle = 20
# Export CSV for sbrio
sbrio_freq = 400
tb_1 = lk.InverseKinematicsPoly(np.array([[px_init], [-0.2]]))
tb_2 = lk.InverseKinematicsPoly(np.array([[mx_init], [-0.2]]))
init_A_tb = tb_1
init_B_tb = tb_1
init_C_tb = tb_2
init_D_tb = tb_2
FSM = FiniteStateMachine(sbrio_freq)
CORGI = Corgi(sbrio_freq, FSM)
CORGI.step_length = L
CORGI.swing_time = T_sw
CORGI.weight_s = C_s
CORGI.weight_u = C_u
CORGI.total_cycle = cycle
CORGI.total_time = total_t
CORGI.setInitPhase(init_A_tb, init_B_tb, init_C_tb, init_D_tb)
CORGI.move(swing_profile=opt_bez)
CORGI.exportCSV(csv_filepath)
print("sbrio exported")

# Export CSV for webot
tb_1 = lk.InverseKinematicsPoly(np.array([[px_init], [-0.2]]))
tb_2 = lk.InverseKinematicsPoly(np.array([[mx_init], [-0.2]]))
init_A_tb = tb_1
init_B_tb = tb_1
init_C_tb = tb_2
init_D_tb = tb_2
FSM = FiniteStateMachine(40)
CORGI = Corgi(40, FSM)
CORGI.step_length = L
CORGI.swing_time = T_sw
CORGI.weight_s = C_s
CORGI.weight_u = C_u
CORGI.total_cycle = cycle
CORGI.total_time = total_t
CORGI.setInitPhase(init_A_tb, init_B_tb, init_C_tb, init_D_tb)
CORGI.move(swing_profile=opt_bez)
CORGI.exportCSV(webot_filepath)
print("sbrio exported")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_21027/2683738682.py", line 11, in <module>
    pickle.dump(min_, f)
NameError: name 'min_' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1396, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1287, in structured_traceback
    return VerboseTB.structured_traceback(
  File "/home/guanlunlu/.local/lib/python3.8/site-packages/